In [300]:

from collections import Counter
import pandas as pd

In [301]:
df = pd.read_csv('/mnt/d/Documents/数学建模国赛/C题目/data/GTD_v2.csv')

/home/nocater/anaconda3/envs/ml/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


去除index列  
定义eventid为index  
替换`related`中的`and`字符，为之后的事件归档做准备

In [302]:
df.columns

df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index('eventid')
df.related = df.related.str.replace('and', ',')
df.related = df.related.str.replace('  ', ' ')
df.head(5)

,iyear,imonth,iday,extended,resolution,crit1,crit2,crit3,doubtterr,alternative,...,ransomamt,ransomamtus,ransompaid,ransompaidus,hostkidoutcome,nreleased,INT_LOG,INT_IDEO,INT_MISC,INT_ANY
eventid,,,,,,,,,,,,,,,,,,,,,
199801010001,1998,1,1,0,NaN,1,1,1,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,1
199801010002,1998,1,1,0,NaN,1,1,1,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-9,-9,0,-9
199801010003,1998,1,1,0,NaN,1,1,1,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,1
199801020001,1998,1,2,0,NaN,1,1,1,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-9,-9,1,1
199801020002,1998,1,2,0,NaN,1,1,1,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-9,-9,0,-9


**建立新字段持续时间先放一放**

In [303]:
df.columns

Index(['iyear', 'imonth', 'iday', 'extended', 'resolution', 'crit1', 'crit2',
       'crit3', 'doubtterr', 'alternative', 'multiple', 'related', 'country',
       'region', 'provstate', 'city', 'vicinity', 'specificity', 'attacktype1',
       'attacktype2', 'attacktype3', 'success', 'suicide', 'weaptype1',
       'weapsubtype1', 'targtype1', 'corp1', 'target1', 'natlty1', 'gname',
       'guncertain1', 'nperps', 'nperpcap', 'claimed', 'claimmode',
       'compclaim', 'claim2', 'nkill', 'nkillus', 'nkillter', 'nwound',
       'nwoundus', 'nwoundte', 'property', 'propextent', 'propvalue',
       'ishostkid', 'nhostkid', 'nhostkidus', 'nhours', 'ndays', 'ransom',
       'ransomamt', 'ransomamtus', 'ransompaid', 'ransompaidus',
       'hostkidoutcome', 'nreleased', 'INT_LOG', 'INT_IDEO', 'INT_MISC',
       'INT_ANY'],
      dtype='object')

# 任务0 对事件进行归类
根据`related`字段，将同一事件进行归档  
定义`add_fields = ['nkill','nwound','nhostkid','ransomamt']`  
将事件的以上信息整合

In [304]:

# 筛选出来 multiple 为1 但realted不为空的准备分组
groups = df[(df.multiple==1)&(df.related.notnull())].related

# print(df[(df.multiple==1)&(df.related.notnull())][['multiple','related']])
print(groups[:1])

# key 是EVENTID values是它关联的EVENTID
dict_ee = {}
for g in groups:
    l = g.split(',')
    l = list(map(lambda x: x.strip(), l))
#     print( l, min(l), dict_ee.get(min(l)))
    if dict_ee.get(min(l)) is None:
        key = min(l)
        l.remove(key)
        dict_ee[key] = l
print(dict_ee.get(str(200109110004)))

eventid
199801040001    199801040001, 199801040002
Name: related, dtype: object
['200109110005', '200109110006', '200109110007']


In [305]:
# t = 199802140003
# print(int(k) not in df.index)

print(len(df))
print(df.loc[[200109110004,200109110005,200109110006,200109110007]][add_fields])

add_fields = ['nkill','nwound','nhostkid','ransomamt']
for k,v in dict_ee.items():
    if int(k) not in df.index:continue
    for f in add_fields:
        for subeventid in v:
            if int(subeventid) not in df.index:continue
            #信息汇总到事件第一个Eventid
            df.loc[int(k), f] += df.loc[int(subeventid)][f]
            # 删除列
            df.drop(int(subeventid), inplace=True)
    
print(df.loc[200109110004][add_fields])
print(len(df))

114183
               nkill  nwound  nhostkid  ransomamt
eventid                                          
200109110004  1384.0  8190.0      88.0        NaN
200109110005  1383.0  8191.0      59.0        NaN
200109110006   190.0   106.0      59.0        NaN
200109110007    44.0     6.0      40.0        NaN
nkill        3001
nwound       8190
nhostkid       88
ransomamt     NaN
Name: 200109110004, dtype: object
102070


In [306]:
print(df.loc[200109110004][add_fields])

nkill        3001
nwound       8190
nhostkid       88
ransomamt     NaN
Name: 200109110004, dtype: object


将归档后的文件写入v3版本  
每个事件只保留第一个`eventid`，并整合了`nkill','nwound','nhostkid','ransomamt'`字敦

In [307]:
df.to_csv('/mnt/d/Documents/数学建模国赛/C题目/data/GTD_v3.csv')

# 任务1 聚类

In [330]:
df = pd.read_csv('/mnt/d/Documents/数学建模国赛/C题目/data/GTD_v3.csv')
df = df.set_index('eventid')

df = df.fillna(0)
fields = [ 'iyear', 'imonth', 'iday', 'resolution', 'related']
df.drop(fields, axis=1, inplace=True)
df.info()

/home/nocater/anaconda3/envs/ml/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 102070 entries, 199801010001 to 201712310032
Data columns (total 57 columns):
extended          102070 non-null int64
crit1             102070 non-null int64
crit2             102070 non-null int64
crit3             102070 non-null int64
doubtterr         102070 non-null float64
alternative       102070 non-null float64
multiple          102070 non-null float64
country           102070 non-null int64
region            102070 non-null int64
provstate         102070 non-null int64
city              102070 non-null int64
vicinity          102070 non-null int64
specificity       102070 non-null float64
attacktype1       102070 non-null int64
attacktype2       102070 non-null float64
attacktype3       102070 non-null float64
success           102070 non-null int64
suicide           102070 non-null int64
weaptype1         102070 non-null int64
weapsubtype1      102070 non-null float64
targtype1         102070 non-null int64
corp1             

In [310]:
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=5, random_state=0).fit(df.values)
label = kmeans.predict(df.values)


In [331]:
# 类别和eventid对应列表
dict_ei = dict(zip(df.index, list(range(len(df.index)))))
dict_ie = dict(zip(list(range(len(df.index))),df.index))


df = df.replace(-9, 0)

# df2 = df.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))

# 主要考虑字段信息 
# propextent 1 灾难损害 2 重大损害 3 较小 4 未知
# 
q1_fields = ['nkill','nwound','nhostkid','targtype1','crit1','crit2','crit3','propextent','ransom','ransomamt']

In [332]:
# 聚类
from sklearn.cluster import MiniBatchKMeans

mbkmeans = MiniBatchKMeans(n_clusters=5, random_state=0).fit(df.values)

r = mbkmeans.predict(df.values)

from collections import Counter
Counter(r)

# 效果还行 Counter({3: 17613, 4: 33885, 2: 47002, 1: 15424, 0: 259})

Counter({2: 51256, 4: 30400, 1: 19898, 3: 255, 0: 261})

In [333]:
df['level'] = r
dict_notolevel = {2:1,4:2,1:3,3:4,0:5}
df.level = list(map(lambda x: dict_notolevel.get(x), df.level))
df.groupby('level')[q1_fields].mean()

,nkill,nwound,nhostkid,targtype1,crit1,crit2,crit3,propextent,ransom,ransomamt
level,,,,,,,,,,
1,2.805701,3.821494,0.637564,8.368815,0.987943,0.995201,0.867294,1.252595,0.003453,1.899230e+01
2,2.761316,3.222105,0.213092,8.486118,0.988651,0.994803,0.872434,1.368158,0.002500,1.147913e+01
3,2.399588,2.884762,0.439743,9.784199,0.990703,0.997035,0.904714,1.357121,0.003518,1.448078e+01
4,1.137255,3.054902,-0.858824,5.568627,0.988235,1.000000,0.996078,2.745098,0.000000,0.000000e+00
5,0.356322,0.310345,4.708812,8.643678,0.931034,1.000000,0.984674,0.421456,1.000000,7.198800e+06


In [334]:
# 十大恐怖袭击事件
nos = [200109110004,200403110003,200409010002,200507070002,200811260010,201003290010,201101240016,201107220012,201403010012,201304150001]
df.loc[nos][(q1_fields+['level'])]

/home/nocater/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  app.launch_new_instance()


,nkill,nwound,nhostkid,targtype1,crit1,crit2,crit3,propextent,ransom,ransomamt,level
eventid,,,,,,,,,,,
200109110004,3001.0,8190.0,88.0,14.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
200403110003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200409010002,344.0,727.0,1200.0,8.0,1.0,1.0,1.0,3.0,0.0,0.0,2.0
200507070002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200811260010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201003290010,40.0,48.0,0.0,19.0,1.0,1.0,1.0,4.0,0.0,0.0,1.0
201101240016,38.0,168.0,0.0,6.0,1.0,1.0,1.0,4.0,0.0,0.0,1.0
201107220012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201403010012,33.0,143.0,0.0,19.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0


In [335]:
# 待预测类别
q1_nos = [200108110012,200511180002,200901170021,201402110015,201405010071,201411070001,201411070002,201412160041,201508010015,201705080012]
df.loc[q1_nos][(q1_fields+['level'])]

/home/nocater/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  app.launch_new_instance()


,nkill,nwound,nhostkid,targtype1,crit1,crit2,crit3,propextent,ransom,ransomamt,level
eventid,,,,,,,,,,,
200108110012,259.0,160.0,0.0,19.0,1.0,1.0,1.0,2.0,0.0,0.0,3.0
200511180002,77.0,90.0,0.0,15.0,1.0,1.0,1.0,2.0,0.0,0.0,3.0
200901170021,400.0,0.0,0.0,15.0,1.0,1.0,1.0,3.0,0.0,0.0,1.0
201402110015,15.0,0.0,8.0,4.0,1.0,1.0,0.0,4.0,0.0,0.0,3.0
201405010071,30.0,0.0,0.0,14.0,1.0,1.0,1.0,4.0,0.0,0.0,3.0
201411070001,6.0,1.0,0.0,17.0,1.0,1.0,1.0,3.0,0.0,0.0,3.0
201411070002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201412160041,150.0,0.0,150.0,14.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
201508010015,1.0,1.0,0.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0,3.0


In [336]:
df['score'] = df.nkill + df.nwound*0.3 + df.nhostkid*0.1
df.sort_values(['score'], ascending=False)[q1_fields+['level','score']].head(10)

,nkill,nwound,nhostkid,targtype1,crit1,crit2,crit3,propextent,ransom,ransomamt,level,score
eventid,,,,,,,,,,,,
200109110004,3001.0,8190.0,88.0,14,1,1,1,1.0,0.0,0.0,1,5466.8
201406150063,1570.0,0.0,1686.0,4,1,1,0,0.0,0.0,0.0,1,1738.6
199808070002,235.0,4000.0,0.0,7,1,1,1,2.0,0.0,0.0,1,1435.0
200708150005,500.0,750.0,0.0,14,1,1,1,2.0,0.0,0.0,1,725.0
201710140002,590.0,316.0,0.0,1,1,1,1,4.0,0.0,0.0,1,684.8
200409010002,344.0,727.0,1200.0,8,1,1,1,3.0,0.0,0.0,2,682.1
201406100041,670.0,0.0,0.0,3,1,1,1,4.0,0.0,0.0,3,670.0
200403210001,518.0,216.0,0.0,2,1,1,1,4.0,0.0,0.0,1,582.8
201408200027,517.0,0.0,150.0,4,1,1,0,4.0,0.0,0.0,1,532.0
